In [1]:
cd /Users/wangshihang/Python/Experiment/MoEL_tensorflow

/Users/wangshihang/Python/Experiment/MoEL_tensorflow


In [ ]:
class Lang:
    def __init__(self, init_index2word):
        self.word2index = {str(v): int(k) for k, v in init_index2word.items()}
        self.word2count = {str(v): 1 for k, v in init_index2word.items()}
        self.index2word = init_index2word 
        self.n_words = len(init_index2word)  # Count default tokens
      
    def index_words(self, sentence):
        for word in sentence:
            self.index_word(word.strip())

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1



def clean(sentence, word_pairs):
    sentence = sentence.lower()
    for k, v in word_pairs.items():
        sentence = sentence.replace(k,v)
    sentence = nltk.word_tokenize(sentence)
    return sentence


def read_langs(vocab):
    word_pairs = {"it's":"it is", "don't":"do not", "doesn't":"does not", "didn't":"did not", "you'd":"you would", "you're":"you are", "you'll":"you will", "i'm":"i am", "they're":"they are", "that's":"that is", "what's":"what is", "couldn't":"could not", "i've":"i have", "we've":"we have", "can't":"cannot", "i'd":"i would", "i'd":"i would", "aren't":"are not", "isn't":"is not", "wasn't":"was not", "weren't":"were not", "won't":"will not", "there's":"there is", "there're":"there are"}
    train_context = np.load('empathetic-dialogue/sys_dialog_texts.train.npy',allow_pickle=True)
    train_target = np.load('empathetic-dialogue/sys_target_texts.train.npy',allow_pickle=True)
    train_emotion = np.load('empathetic-dialogue/sys_emotion_texts.train.npy',allow_pickle=True)
    train_situation = np.load('empathetic-dialogue/sys_situation_texts.train.npy',allow_pickle=True)

    dev_context = np.load('empathetic-dialogue/sys_dialog_texts.dev.npy',allow_pickle=True)
    dev_target = np.load('empathetic-dialogue/sys_target_texts.dev.npy',allow_pickle=True)
    dev_emotion = np.load('empathetic-dialogue/sys_emotion_texts.dev.npy',allow_pickle=True)
    dev_situation = np.load('empathetic-dialogue/sys_situation_texts.dev.npy',allow_pickle=True)
    
    test_context = np.load('empathetic-dialogue/sys_dialog_texts.test.npy',allow_pickle=True)
    test_target = np.load('empathetic-dialogue/sys_target_texts.test.npy',allow_pickle=True)
    test_emotion = np.load('empathetic-dialogue/sys_emotion_texts.test.npy',allow_pickle=True)
    test_situation = np.load('empathetic-dialogue/sys_situation_texts.test.npy',allow_pickle=True)

    data_train = {'context':[],'target':[],'emotion':[], 'situation':[]}
    data_dev = {'context':[],'target':[],'emotion':[], 'situation':[]}
    data_test = {'context':[],'target':[],'emotion':[], 'situation':[]}

    for context in train_context:
        u_list = []
        for u in context:
            u = clean(u, word_pairs)
            u_list.append(u)
            vocab.index_words(u)
        data_train['context'].append(u_list)
    for target in train_target:
        target = clean(target, word_pairs)
        data_train['target'].append(target)
        vocab.index_words(target)
    for situation in train_situation:
        situation = clean(situation, word_pairs)
        data_train['situation'].append(situation)
        vocab.index_words(situation)
    for emotion in train_emotion:
        data_train['emotion'].append(emotion)
    assert len(data_train['context']) == len(data_train['target']) == len(data_train['emotion']) == len(data_train['situation'])

    for context in dev_context:
        u_list = []
        for u in context:
            u = clean(u, word_pairs)
            u_list.append(u)
            vocab.index_words(u)
        data_dev['context'].append(u_list)
    for target in dev_target:
        target = clean(target, word_pairs)
        data_dev['target'].append(target)
        vocab.index_words(target)
    for situation in dev_situation:
        situation = clean(situation, word_pairs)
        data_dev['situation'].append(situation)
        vocab.index_words(situation)
    for emotion in dev_emotion:
        data_dev['emotion'].append(emotion)
    assert len(data_dev['context']) == len(data_dev['target']) == len(data_dev['emotion']) == len(data_dev['situation'])

    for context in test_context:
        u_list = []
        for u in context:
            u = clean(u, word_pairs)
            u_list.append(u)
            vocab.index_words(u)
        data_test['context'].append(u_list)
    for target in test_target:
        target = clean(target, word_pairs)
        data_test['target'].append(target)
        vocab.index_words(target)
    for situation in test_situation:
        situation = clean(situation, word_pairs)
        data_test['situation'].append(situation)
        vocab.index_words(situation)
    for emotion in test_emotion:
        data_test['emotion'].append(emotion)
    assert len(data_test['context']) == len(data_test['target']) == len(data_test['emotion']) == len(data_test['situation'])
    return data_train, data_dev, data_test, vocab

#load_dataset returns 3 dictionaries data_tra, data_val, data_tst and vocab <__main__.Lang at 0x7fdeaf8be0d0>

def load_dataset():
    if(os.path.exists('empathetic-dialogue/dataset_preproc.p')):
        print("LOADING empathetic_dialogue")
        with open('empathetic-dialogue/dataset_preproc.p', "rb") as f:
            [data_tra, data_val, data_tst, vocab] = pickle.load(f)
    else:
        print("Building dataset...")
        data_tra, data_val, data_tst, vocab  = read_langs(vocab=Lang({0: "UNK", 1: "PAD", 2: "EOS", 3: "SOS",
                                                                      4:"USR", 5:"SYS", 6:"CLS"})) 
        with open('empathetic-dialogue/dataset_preproc.p', "wb") as f:
            pickle.dump([data_tra, data_val, data_tst, vocab], f)
            print("Saved PICKLE")
    for i in range(3):
        print('[situation]:', ' '.join(data_tra['situation'][i]))
        print('[emotion]:', data_tra['emotion'][i])
        print('[context]:', [' '.join(u) for u in data_tra['context'][i]])
        print('[target]:', ' '.join(data_tra['target'][i]))
        print(" ")
    return data_tra, data_val, data_tst, vocab

In [4]:
import os
import nltk
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import array_ops

tf.random.set_seed(1)

pairs_tra, pairs_val, pairs_tst, vocab = load_dataset()

# data: 4-key dictionary
def process_item(data):
    """Returns one data pair (source and target)."""
    emo_map = {
        'surprised': 0, 'excited': 1, 'annoyed': 2, 'proud': 3, 'angry': 4, 'sad': 5, 'grateful': 6, 'lonely': 7,
        'impressed': 8, 'afraid': 9, 'disgusted': 10, 'confident': 11, 'terrified': 12, 'hopeful': 13, 'anxious': 14, 'disappointed': 15,
        'joyful': 16, 'prepared': 17, 'guilty': 18, 'furious': 19, 'nostalgic': 20, 'jealous': 21, 'anticipating': 22, 'embarrassed': 23,
        'content': 24, 'devastated': 25, 'sentimental': 26, 'caring': 27, 'trusting': 28, 'ashamed': 29, 'apprehensive': 30, 'faithful': 31}
    ls_item = []
    for index in range(0, len(data['context'])):
        
        item = {}
        item["context_text"] = data["context"][index]
        item["target_text"] = data["target"][index]
        item["emotion_text"] = data["emotion"][index]

        item["context"], item["context_mask"] = preprocess(item["context_text"])

        item["target"] = preprocess(item["target_text"], anw=True)
        item["emotion"], item["emotion_label"] = preprocess_emo(item["emotion_text"], emo_map)
        ls_item.append(item)
    
    ls_item.sort(key=lambda x: len(x["context"]), reverse=True)
    

    item_info = {} #item_info is a 8 key dictionary of lists
    for key in ls_item[0].keys():
        item_info[key] = [d[key] for d in ls_item]
    #print('CONTENT OF item_info context:', item_info['context'])
    return item_info #dict of lists

def preprocess(arr, anw=False):
    """Converts words to ids."""
    if(anw):
        sequence = [vocab.word2index[word] if word in vocab.word2index else 0 for word in arr] + [2]
        return sequence
    else:
        X_dial = [6]
        X_mask = [6]
    for i, sentence in enumerate(arr):
            X_dial += [vocab.word2index[word] if word in vocab.word2index else 0 for word in sentence] #修改
            spk = vocab.word2index["USR"] if i % 2 == 0 else vocab.word2index["SYS"]
            X_mask += [spk for _ in range(len(sentence))] #修改
    assert len(X_dial) == len(X_mask)

    return X_dial, X_mask

def preprocess_emo(emotion, emo_map):
    program = [0]*len(emo_map)
    program[emo_map[emotion]] = 1

    return program, emo_map[emotion]

   
#each item has 7 tensors
def convert_to_dataset(item_info,batch_size = 16):
    """convert to Tensorflow data.DataSet"""
    def _element_length_fn(a,b,c,d,e,f,g):
         return array_ops.shape(a)[0]
        
    def generator():
    #dt = [data_tst.__getitem__(i) for i in range(0, 30)]
        for i in range(0, len(item_info['context'])):
            # creates x’s and y’s for the dataset.
            input_batch = item_info['context'][i] #var len [...]
            input_length = [len(item_info['context'][i])] #修改
            mask_input = item_info['context_mask'][i] #var len [...]
            target_batch = item_info['target'][i] #var len [...]
            target_length = [len(item_info['target'][i])] #[int]
            target_program = item_info['emotion'][i] #var len [...]
            program_label = [item_info['emotion_label'][i]] #[int]
            yield input_batch, input_length, mask_input, target_batch, target_length, target_program, program_label
    
    toy_dataset = tf.data.Dataset.from_generator(
         generator,
         output_signature=(
             tf.TensorSpec(shape=([None]), dtype=tf.int32),
             tf.TensorSpec(shape=(1,), dtype=tf.int32),
             tf.TensorSpec(shape=([None]), dtype=tf.int32),
             tf.TensorSpec(shape=([None]), dtype=tf.int32),
             tf.TensorSpec(shape=(1,), dtype=tf.int32),
             tf.TensorSpec(shape=([None]), dtype=tf.int32),
             tf.TensorSpec(shape=(1,), dtype=tf.int32)
         ))
    #print('LEN CONTEXT',len(item_info['context']))
    bound_num = len(item_info['context']) // batch_size
    
    #print('BOUND NUM',bound_num)
    #let's use context length for bucket_by_sequence_length
    toy_boundaries = [len(x) for i, x in enumerate(item_info['context']) if i% bound_num == 0]
    #[batch_size] * len(bucket_boundaries) + 1
    toy_bucket_batch_sizes = [batch_size] * (len(toy_boundaries) + 1)

    #this is similar to the "merge" function to pad lists for each batch
    dataset = toy_dataset.apply(tf.data.experimental.bucket_by_sequence_length(_element_length_fn, 
                                                                           bucket_boundaries = toy_boundaries,
                                                                           bucket_batch_sizes = toy_bucket_batch_sizes,
                                                                           drop_remainder=True,
                                                                           #padding_values=1,
                                                                           pad_to_bucket_boundary=False))
    return dataset

def prepare_data_seq(batch_size):
    pairs_tra, pairs_val, pairs_tst, vocab = load_dataset()
    
    emo_map = {
        'surprised': 0, 'excited': 1, 'annoyed': 2, 'proud': 3, 'angry': 4, 'sad': 5, 'grateful': 6, 'lonely': 7,
        'impressed': 8, 'afraid': 9, 'disgusted': 10, 'confident': 11, 'terrified': 12, 'hopeful': 13, 'anxious': 14, 'disappointed': 15,
        'joyful': 16, 'prepared': 17, 'guilty': 18, 'furious': 19, 'nostalgic': 20, 'jealous': 21, 'anticipating': 22, 'embarrassed': 23,
        'content': 24, 'devastated': 25, 'sentimental': 26, 'caring': 27, 'trusting': 28, 'ashamed': 29, 'apprehensive': 30, 'faithful': 31}
    
    item_info_tra = process_item(pairs_tra)
    item_info_val = process_item(pairs_val)
    item_info_tst = process_item(pairs_tst)
    
    data_loader_tra = convert_to_dataset(item_info_tra)
    data_loader_val = convert_to_dataset(item_info_val)
    data_loader_tst = convert_to_dataset(item_info_tst)
    
    
    return data_loader_tra, data_loader_val, data_loader_tst, vocab, len(emo_map)


def test():
    pairs_tra, pairs_val, pairs_tst, vocab = load_dataset()
    
    item_info_tra = process_item(pairs_tra)
    item_info_val = process_item(pairs_val)
    item_info_tst = process_item(pairs_tst)
    
    data_loader_tra = convert_to_dataset(item_info_tra)
    data_loader_val = convert_to_dataset(item_info_val)
    data_loader_tst = convert_to_dataset(item_info_tst)
    #vocab, len(dataset_train.emo_map)
    
    return data_loader_tra, data_loader_tst

Building dataset...
Saved PICKLE
[situation]: i remember going to the fireworks with my best friend . there was a lot of people , but it only felt like us in the world .
[emotion]: sentimental
[context]: ['i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world .']
[target]: was this a friend you were in love with , or just a best friend ?
 
[situation]: i remember going to the fireworks with my best friend . there was a lot of people , but it only felt like us in the world .
[emotion]: sentimental
[context]: ['i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world .', 'was this a friend you were in love with , or just a best friend ?', 'this was a best friend . i miss her .']
[target]: where has she gone ?


In [5]:
data_loader_tra, data_loader_val, data_loader_tst, vocab, program_number = prepare_data_seq(batch_size=16)

LOADING empathetic_dialogue
[situation]: i remember going to the fireworks with my best friend . there was a lot of people , but it only felt like us in the world .
[emotion]: sentimental
[context]: ['i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world .']
[target]: was this a friend you were in love with , or just a best friend ?
 
[situation]: i remember going to the fireworks with my best friend . there was a lot of people , but it only felt like us in the world .
[emotion]: sentimental
[context]: ['i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world .', 'was this a friend you were in love with , or just a best friend ?', 'this was a best friend . i miss her .']
[target]: where has she gone ?
 
[si

In [6]:
def make_infinite(dataloader):
    while True:
        for x in dataloader:
            yield x
data_iter = make_infinite(data_loader_tra)

In [7]:
count=0
for i in data_iter:
    for k in i:
        print(k.shape)
    count+=1
    print('...........')
    if count > 10:
        break

(16, 334)
(16, 1)
(16, 334)
(16, 72)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 209)
(16, 1)
(16, 209)
(16, 100)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 192)
(16, 1)
(16, 192)
(16, 82)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 182)
(16, 1)
(16, 182)
(16, 36)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 171)
(16, 1)
(16, 171)
(16, 114)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 164)
(16, 1)
(16, 164)
(16, 85)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 161)
(16, 1)
(16, 161)
(16, 38)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 156)
(16, 1)
(16, 156)
(16, 103)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 151)
(16, 1)
(16, 151)
(16, 51)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 148)
(16, 1)
(16, 148)
(16, 78)
(16, 1)
(16, 32)
(16, 1)
...........
(16, 145)
(16, 1)
(16, 145)
(16, 85)
(16, 1)
(16, 32)
(16, 1)
...........


In [204]:
import os
import math
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras import layers
import matplotlib.pyplot as plt

class EncoderLayer(layers.Layer):
    """
    Represents one Encoder layer of the Transformer Encoder
    Refer Fig. 1 in https://arxiv.org/pdf/1706.03762.pdf
    NOTE: The layer normalization step has been moved to the input as per latest version of T2T
    """
    def __init__(self, hidden_size, total_key_depth, total_value_depth, filter_size, num_heads,
                 bias_mask=None, layer_dropout=0.0, attention_dropout=0.0, relu_dropout=0.0):
        """
        Parameters:
            hidden_size: Hidden size
            total_key_depth: Size of last dimension of keys. Must be divisible by num_head
            total_value_depth: Size of last dimension of values. Must be divisible by num_head
            output_depth: Size last dimension of the final output
            filter_size: Hidden size of the middle layer in FFN
            num_heads: Number of attention heads
            bias_mask: Masking tensor to prevent connections to future elements
            layer_dropout: Dropout for this layer
            attention_dropout: Dropout probability after attention (Should be non-zero only during training)
            relu_dropout: Dropout probability after relu in FFN (Should be non-zero only during training)
        """
        super(EncoderLayer, self).__init__()
        
        self.multi_head_attention = MultiHeadAttention(hidden_size, total_key_depth, total_value_depth, 
                                                       hidden_size, num_heads, bias_mask, attention_dropout)
        
        self.positionwise_feed_forward = PositionwiseFeedForward(hidden_size, filter_size, hidden_size,
                                                                 layer_config='cc', padding = 'both', 
                                                                 dropout=relu_dropout)
        self.dropout = layers.Dropout(layer_dropout)
        self.layer_norm_mha = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm_ffn = layers.LayerNormalization(epsilon=1e-6)

    def __call__(self, inputs, mask=None, training=True):
        x = inputs
        
        # Layer Normalization
        x_norm = self.layer_norm_mha(x)
        # Multi-head attention
        y, _ = self.multi_head_attention(x_norm, x_norm, x_norm, mask)
        # Dropout and residual
        x = self.dropout(x + y, training=training)
        # Layer Normalization
        x_norm = self.layer_norm_ffn(x)
        # Positionwise Feedforward
        y = self.positionwise_feed_forward(x_norm)
        # Dropout and residual
        y = self.dropout(x + y, training=training)
        # y = self.layer_norm_end(y)
        return y
    
class DecoderLayer(layers.Layer):
    """
    Represents one Decoder layer of the Transformer Decoder
    Refer Fig. 1 in https://arxiv.org/pdf/1706.03762.pdf
    NOTE: The layer normalization step has been moved to the input as per latest version of T2T
    """
    def __init__(self, hidden_size, total_key_depth, total_value_depth, filter_size, num_heads,
                 bias_mask, layer_dropout=0.0, attention_dropout=0.0, relu_dropout=0.0):
        """
        Parameters:
            hidden_size: Hidden size
            total_key_depth: Size of last dimension of keys. Must be divisible by num_head
            total_value_depth: Size of last dimension of values. Must be divisible by num_head
            output_depth: Size last dimension of the final output
            filter_size: Hidden size of the middle layer in FFN
            num_heads: Number of attention heads
            bias_mask: Masking tensor to prevent connections to future elements
            layer_dropout: Dropout for this layer
            attention_dropout: Dropout probability after attention (Should be non-zero only during training)
            relu_dropout: Dropout probability after relu in FFN (Should be non-zero only during training)
        """ 
        super(DecoderLayer, self).__init__()
        self.multi_head_attention_dec = MultiHeadAttention(hidden_size, total_key_depth, total_value_depth, 
                                                       hidden_size, num_heads, bias_mask, attention_dropout)

        self.multi_head_attention_enc_dec = MultiHeadAttention(hidden_size, total_key_depth, total_value_depth, 
                                                       hidden_size, num_heads, None, attention_dropout)
        
        self.positionwise_feed_forward = PositionwiseFeedForward(hidden_size, filter_size, hidden_size,
                                                                 layer_config='cc', padding = 'left', 
                                                                 dropout=relu_dropout)
        self.dropout = layers.Dropout(layer_dropout)
        self.layer_norm_mha_dec = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm_mha_enc = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm_ffn = layers.LayerNormalization(epsilon=1e-6)
        # self.layer_norm_end = layers.LayerNormalization(epsilon=1e-6)
    
    def __call__(self, inputs, training=True):
        """
        NOTE: Inputs is a tuple consisting of decoder inputs and encoder output
        """

        x, encoder_outputs, attention_weight, mask = inputs
        mask_src, dec_mask = mask
        
        # Layer Normalization before decoder self attention
        x_norm = self.layer_norm_mha_dec(x)
        # Masked Multi-head attention
        y, _ = self.multi_head_attention_dec(x_norm, x_norm, x_norm, dec_mask)
        # Dropout and residual after self-attention
        x = self.dropout(x + y, training=training)
        # Layer Normalization before encoder-decoder attention
        x_norm = self.layer_norm_mha_enc(x)
        # Multi-head encoder-decoder attention
        y, attention_weight = self.multi_head_attention_enc_dec(x_norm, encoder_outputs, encoder_outputs, mask_src)
        # Dropout and residual after encoder-decoder attention
        x = self.dropout(x + y, training=training)
        # Layer Normalization
        x_norm = self.layer_norm_ffn(x)
        # Positionwise Feedforward
        y = self.positionwise_feed_forward(x_norm)
        # Dropout and residual after positionwise feed forward layer
        y = self.dropout(x + y, training=training)
        # y = self.layer_norm_end(y)
        # Return encoder outputs as well to work with tf.keras.models.Sequential
        return y, encoder_outputs, attention_weight, mask

class MultiExpertMultiHeadAttention(layers.Layer):
    def __init__(self, num_experts, input_depth, total_key_depth, total_value_depth, output_depth, 
                 num_heads, bias_mask=None, dropout=0.0):
        """
        Parameters:
            expert_num: Number of experts
            input_depth: Size of last dimension of input
            total_key_depth: Size of last dimension of keys. Must be divisible by num_head
            total_value_depth: Size of last dimension of values. Must be divisible by num_head
            output_depth: Size last dimension of the final output
            num_heads: Number of attention heads
            bias_mask: Masking tensor to prevent connections to future elements
            dropout: Dropout probability (Should be non-zero only during training)
        """
        super(MultiExpertMultiHeadAttention, self).__init__()
        # Checks borrowed from 
        # https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
        if total_key_depth % num_heads != 0:
            print("Key depth (%d) must be divisible by the number of "
                             "attention heads (%d)." % (total_key_depth, num_heads))
            total_key_depth = total_key_depth - (total_key_depth % num_heads)
        if total_value_depth % num_heads != 0:
            print("Value depth (%d) must be divisible by the number of "
                             "attention heads (%d)." % (total_value_depth, num_heads))
            total_value_depth = total_value_depth - (total_value_depth % num_heads)
        self.num_experts = num_experts
        self.num_heads = num_heads
        self.query_scale = (total_key_depth//num_heads)**-0.5 ## sqrt
        self.bias_mask = bias_mask
        
        # Key and query depth will be same
        self.query_linear = layers.Dense(total_key_depth*num_experts, use_bias=False)
        self.key_linear = layers.Dense(total_key_depth*num_experts, use_bias=False)
        self.value_linear = layers.Dense(total_value_depth*num_experts, use_bias=False)
        self.output_linear = layers.Dense(output_depth*num_experts, use_bias=False)
        
        self.dropout = layers.Dropout(dropout)
    
    def _split_heads(self, x):
        """
        Split x such to add an extra num_heads dimension
        Input:
            x: a Tensor with shape [batch_size, seq_length, depth]
        Returns:
            A Tensor with shape [batch_size, num_experts ,num_heads, seq_length, depth/(num_heads*num_experts)]
        """
        if len(x.shape) != 3:
            raise ValueError("x must have rank 3")
        shape = x.shape
        return tf.transpose(tf.reshape(x, [shape[0], shape[1], self.num_experts, self.num_heads, 
                               shape[2]//(self.num_heads*self.num_experts)]), perm=[0, 2, 3, 1, 4])
    
    def _merge_heads(self, x):
        """
        Merge the extra num_heads into the last dimension
        Input:
            x: a Tensor with shape [batch_size, num_experts ,num_heads, seq_length, depth/num_heads]
        Returns:
            A Tensor with shape [batch_size, seq_length, num_experts, depth/num_experts]
        """
        if len(x.shape) != 5:
            raise ValueError("x must have rank 5")
        shape = x.shape
        return tf.reshape(tf.transpose(x, perm=[0, 3, 1, 2, 4]), [shape[0], shape[3], self.num_experts, shape[4]*self.num_heads])
    
    def __call__(self, queries, keys, values, mask):
        
        # Do a linear for each component
        queries = self.query_linear(queries)
        keys = self.key_linear(keys)
        values = self.value_linear(values)
        # Split into multiple heads
        queries = self._split_heads(queries)
        keys = self._split_heads(keys)
        values = self._split_heads(values)
        # Scale queries
        queries *= self.query_scale
        # Combine queries and keys
        logits = tf.matmul(queries, keys, transpose_b=True)
        
        if mask is not None:
            #logits += (mask * -1e18)
            mask = tf.expand_dims(tf.expand_dims(mask, 1), 1)
            logits = logits*(1 - tf.cast(mask, tf.float32))+ (-1e18) * tf.cast(mask, tf.float32)
            

        ## attention weights 
        # attetion_weights = logits.sum(dim=1)/self.num_heads
        # Convert to probabilites
        weights = tf.nn.softmax(logits, axis=-1)
        # Dropout
        weights = self.dropout(weights)
        # Combine with values to get context
        contexts = tf.matmul(weights, values)
        # Merge heads
        contexts = self._merge_heads(contexts)
        #contexts = torch.tanh(contexts)
        # Linear to get output
        outputs = self.output_linear(contexts)
        return outputs

class MultiHeadAttention(layers.Layer):
    """
    Multi-head attention as per https://arxiv.org/pdf/1706.03762.pdf
    Refer Figure 2
    """
    def __init__(self, input_depth, total_key_depth, total_value_depth, output_depth, 
                 num_heads, bias_mask=None, dropout=0.0):
        """
        Parameters:
            input_depth: Size of last dimension of input
            total_key_depth: Size of last dimension of keys. Must be divisible by num_head
            total_value_depth: Size of last dimension of values. Must be divisible by num_head
            output_depth: Size last dimension of the final output
            num_heads: Number of attention heads
            bias_mask: Masking tensor to prevent connections to future elements
            dropout: Dropout probability (Should be non-zero only during training)
        """
        super(MultiHeadAttention, self).__init__()
        # Checks borrowed from 
        # https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py
        # if total_key_depth % num_heads != 0:
        #     raise ValueError("Key depth (%d) must be divisible by the number of "
        #                      "attention heads (%d)." % (total_key_depth, num_heads))
        # if total_value_depth % num_heads != 0:
        #     raise ValueError("Value depth (%d) must be divisible by the number of "
        #                      "attention heads (%d)." % (total_value_depth, num_heads))
            
        if total_key_depth % num_heads != 0:
            print("Key depth (%d) must be divisible by the number of "
                             "attention heads (%d)." % (total_key_depth, num_heads))
            total_key_depth = total_key_depth - (total_key_depth % num_heads)
        if total_value_depth % num_heads != 0:
            print("Value depth (%d) must be divisible by the number of "
                             "attention heads (%d)." % (total_value_depth, num_heads))
            total_value_depth = total_value_depth - (total_value_depth % num_heads)

        self.num_heads = num_heads
        self.query_scale = (total_key_depth//num_heads)**-0.5 ## sqrt
        self.bias_mask = bias_mask
        
        # Key and query depth will be same
        self.query_linear = layers.Dense(total_key_depth, use_bias=False)
        self.key_linear = layers.Dense(total_key_depth, use_bias=False)
        self.value_linear = layers.Dense(total_value_depth, use_bias=False)
        self.output_linear = layers.Dense(output_depth, use_bias=False)
        self.dropout = layers.Dropout(dropout)
    
    def _split_heads(self, x):
        """
        Split x such to add an extra num_heads dimension
        Input:
            x: a Tensor with shape [batch_size, seq_length, depth]
        Returns:
            A Tensor with shape [batch_size, num_heads, seq_length, depth/num_heads]
        """
        if len(x.shape) != 3:
            raise ValueError("x must have rank 3")
        shape = x.shape
        return tf.transpose(tf.reshape(x, [shape[0], shape[1], self.num_heads, shape[2]//self.num_heads]), perm=[0, 2, 1, 3])
    
    def _merge_heads(self, x):
        """
        Merge the extra num_heads into the last dimension
        Input:
            x: a Tensor with shape [batch_size, num_heads, seq_length, depth/num_heads]
        Returns:
            A Tensor with shape [batch_size, seq_length, depth]
        """
        if len(x.shape) != 4:
            raise ValueError("x must have rank 4")
        shape = x.shape
        return tf.reshape(tf.transpose(x, perm=[0, 2, 1, 3]), [shape[0], shape[2], shape[3]*self.num_heads])
    
    def __call__(self, queries, keys, values, mask):
        
        # Do a linear for each component
        queries = self.query_linear(queries)
        keys = self.key_linear(keys)
        values = self.value_linear(values)
        # Split into multiple heads
        queries = self._split_heads(queries)
        keys = self._split_heads(keys)
        values = self._split_heads(values)
        # Scale queries
        queries *= self.query_scale
        # Combine queries and keys
        logits = tf.matmul(queries, keys, transpose_b=True)
        
        if mask is not None:
            #logits += (mask * -1e18)
            mask = tf.expand_dims(mask, 1)
            logits = logits*(1 - tf.cast(mask, tf.float32))+ (-1e18) * tf.cast(mask, tf.float32)

        ## attention weights 
        attetion_weights = tf.reduce_sum(logits, axis=1)/self.num_heads
        # Convert to probabilites
        weights = tf.nn.softmax(logits, axis=-1)
        # Dropout
        weights = self.dropout(weights)
        # Combine with values to get context
        contexts = tf.matmul(weights, values)
        # Merge heads
        contexts = self._merge_heads(contexts)
        #contexts = torch.tanh(contexts)
        # Linear to get output
        outputs = self.output_linear(contexts)       
        return outputs, attetion_weights

class Conv(layers.Layer):
    """
    Convenience class that does padding and convolution for inputs in the format
    [batch_size, sequence length, hidden size]
    """
    def __init__(self, output_size, kernel_size, pad_type):
        """
        Parameters:
            input_size: Input feature size
            output_size: Output feature size
            kernel_size: Kernel width
            pad_type: left -> pad on the left side (to mask future data), 
                      both -> pad on both sides
        """
        super(Conv, self).__init__()
        padding = [kernel_size - 1, 0] if pad_type == 'left' else [kernel_size//2, (kernel_size - 1)//2]
        self.padding = [[0, 0], [0, 0], padding]
        self.output_size = output_size
        self.kernel_size = kernel_size

    def __call__(self, inputs, training=True):
        inputs = tf.pad(tf.transpose(inputs, perm=[0, 2, 1]), self.padding, "CONSTANT") 
        outputs = layers.Conv1D(self.output_size, self.kernel_size, padding='valid')(tf.transpose(inputs, [0, 2, 1]), training=training)
        return outputs

class PositionwiseFeedForward(layers.Layer):
    """
    Does a Linear + RELU + Linear on each of the timesteps
    """
    def __init__(self, input_depth, filter_size, output_depth, layer_config='ll', padding='left', dropout=0.0):
        """
        Parameters:
            input_depth: Size of last dimension of input
            filter_size: Hidden size of the middle layer
            output_depth: Size last dimension of the final output
            layer_config: ll -> linear + ReLU + linear
                          cc -> conv + ReLU + conv etc.
            padding: left -> pad on the left side (to mask future data), 
                     both -> pad on both sides
            dropout: Dropout probability (Should be non-zero only during training)
        """
        super(PositionwiseFeedForward, self).__init__()
        
        self.layer_config = layer_config
        self.linear_layer1 = layers.Dense(filter_size)
        self.linear_layer2 = layers.Dense(output_depth)
        self.conv_layer1 = Conv(filter_size, kernel_size=3, pad_type=padding)
        self.conv_layer2 = Conv(output_depth, kernel_size=3, pad_type=padding)
        self.relu_layer = layers.ReLU()
        self.dropout_layer = layers.Dropout(dropout)
        
    def __call__(self, inputs):
        x = inputs
        for i, cur_layer in enumerate(self.layer_config):
            if i < len(self.layer_config)-1:
                if cur_layer == 'l':
                    x = self.linear_layer1(x)
                elif cur_layer == 'c':
                    x = self.conv_layer1(x)
                else:
                    raise ValueError("Unknown layer type {}".format(cur_layer))
            else:
                if cur_layer == 'l':
                    x = self.linear_layer2(x)
                elif cur_layer == 'c':
                    x = self.conv_layer2(x)
                else:
                    raise ValueError("Unknown layer type {}".format(cur_layer))
            x = self.relu_layer(x)
            x = self.dropout_layer(x)
        return x

def _gen_bias_mask(max_length):
    """
    Generates bias values (-Inf) to mask future timesteps during attention
    """
    np_mask = np.triu(np.full([max_length, max_length], -np.inf), 1)
    tf_mask = tf.cast(np_mask, dtype=tf.float32)
    return tf.expand_dims(tf.expand_dims(tf_mask, 0), 1)

def _gen_timing_signal(length, channels, min_timescale=1.0, max_timescale=1.0e4):
    position = np.arange(length)
    num_timescales = channels // 2
    log_timescale_increment = ( math.log(float(max_timescale) / float(min_timescale)) / (float(num_timescales) - 1))
    inv_timescales = min_timescale * np.exp(np.arange(num_timescales).astype(np.float) * -log_timescale_increment)
    scaled_time = np.expand_dims(position, 1) * np.expand_dims(inv_timescales, 0)

    signal = np.concatenate([np.sin(scaled_time), np.cos(scaled_time)], axis=1)
    signal = np.pad(signal, [[0, 0], [0, channels % 2]], 'constant', constant_values=[0.0, 0.0])
    signal = tf.reshape(signal, [1,length, channels])
    #signal =  signal.reshape([1, length, channels])
    return tf.cast(signal, dtype=tf.float32)

def _get_attn_subsequent_mask(size):
    """
    Get an attention mask to avoid using the subsequent info.
    Args:
        size: int
    Returns:
        (`LongTensor`):
        * subsequent_mask `[1 x size x size]`
    """
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    subsequent_mask = tf.cast(subsequent_mask, dtype=tf.uint8)
    return subsequent_mask

class OutputLayer(layers.Layer):
    """
    Abstract base class for output layer. 
    Handles projection to output labels
    """
    def __init__(self, hidden_size, output_size):
        super(OutputLayer, self).__init__()
        self.output_size = output_size
        self.output_projection = layers.Dense(output_size)

    def loss(self, hidden, labels):
        raise NotImplementedError('Must implement {}.loss'.format(self.__class__.__name__))

class SoftmaxOutputLayer(OutputLayer):
    """
    Implements a softmax based output layer
    """
    def forward(self, hidden):
        logits = self.output_projection(hidden)
        probs = tf.nn.softmax(logits, axis=-1)
        predictions = tf.math.argmax(probs, axis=-1)
        return predictions

    def loss(self, hidden, labels):
        logits = self.output_projection(hidden)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        return -tf.math.reduce_sum(tf.one_hot(labels, depth=self.output_size)*tf.nn.log_softmax(log_probs, -1))/len(labels)

def position_encoding(sentence_size, embedding_dim):
    encoding = np.ones((embedding_dim, sentence_size), dtype=np.float32)
    ls = sentence_size + 1
    le = embedding_dim + 1
    for i in range(1, le):
        for j in range(1, ls):
            encoding[i-1, j-1] = (i - (embedding_dim+1)/2) * (j - (sentence_size+1)/2)
    encoding = 1 + 4 * encoding / embedding_dim / sentence_size
    # Make position encoding of time words identity to avoid modifying them
    # encoding[:, -1] = 1.0
    return np.transpose(encoding)

def gen_embeddings(vocab):
    """
        Generate an initial embedding matrix for `word_dict`.
        If an embedding file is not given or a word is not in the embedding file,
        a randomly initialized vector will be used.
    """
    embeddings = np.random.randn(vocab.n_words, 100) * 0.01 
    print('Embeddings: %d x %d' % (vocab.n_words, 100))
    return embeddings

# class TF_Embedding(layers.Layer):
#     def __init__(self, vocab, d_model, padding_idx=0, pretrain=True, **kwargs):
#         super(TF_Embedding, self).__init__(**kwargs)
#         self.input_dim = vocab.n_words
#         self.output_dim = d_model
#         self.padding_idx = padding_idx
#         if pretrain:
#             pre_embedding = gen_embeddings(vocab)
#             self.embeddings =  layers.Embedding(self.input_dim, self.output_dim, weights=[pre_embedding])
#         else:
#             self.embeddings = self.add_weight(
#                 shape=(self.input_dim, self.output_dim),
#                 initializer='random_normal',
#                 dtype='float32')

#     def call(self, inputs): 
#         def compute_mask():
#             return tf.not_equal(inputs, self.padding_idx)
        
#         out = tf.nn.embedding_lookup(self.embeddings, inputs)
#         masking = compute_mask() # [B, T], bool
#         masking = tf.cast(tf.tile(masking[:,:, tf.newaxis], [1,1,self.output_dim]), 
#                           dtype=tf.float32) # [B, T, D]
#         return tf.multiply(out, masking)

# class Embeddings(layers.Layer):
#     def __init__(self, vocab, d_model, padding_idx=None, pretrain=True):
#         super(Embeddings, self).__init__()
#         self.lut = TF_Embedding(vocab, d_model, padding_idx=padding_idx, pretrain=pretrain)
#         self.d_model = d_model

#     def forward(self, x):
#         return self.lut(x) * math.sqrt(self.d_model)

# def share_embedding(vocab, pretrain=False):#修改
#     embedding = Embeddings(vocab, 100, padding_idx=1, pretrain=pretrain)
#     #embedding.lut.weight.data.copy_(torch.FloatTensor(pre_embedding))
#     #embedding.lut.weight.data.requires_grad = True
#     return embedding

#修改
class Embeddinglayer(layers.Layer):
    def __init__(self, vocab_size, d_model):
        # model hyper parameter variables
        super(Embeddinglayer, self).__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.embedding = layers.Embedding(vocab_size, d_model)

    def __call__(self, sequences):
        output = self.embedding(sequences) * tf.sqrt(tf.cast(self.d_model, dtype=tf.float32))
        return output

class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
    
    def state_dict(self):
        return self.optimizer.state_dict()

    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))

def get_attn_key_pad_mask(seq_k, seq_q):
    ''' For masking out the padding part of key sequence. '''

    # Expand to fit the shape of key query attention matrix.
    len_q = seq_q.shape[1]
    padding_mask = tf.math.equal(seq_k, 1)
    padding_mask = tf.broadcast_to(tf.expand_dims(padding_mask, 1), [-1, len_q, -1]) # b x lq x lk
    return padding_mask

def get_input_from_batch(batch):
    enc_batch = batch[0] #enc_batch = batch["input_batch"]
    enc_lens = batch[1] # enc_lens = batch["input_lengths"]
    batch_size, max_enc_len = enc_batch.shape
    assert len(enc_lens) == batch_size

    enc_padding_mask = tf.cast(sequence_mask(enc_lens, max_len=max_enc_len), dtype=tf.float32)

    extra_zeros = None
    enc_batch_extend_vocab = None

    c_t_1 = tf.zeros((batch_size, 2 * 100))

    coverage = None

    return enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, c_t_1, coverage

def get_output_from_batch(batch):
    #dec_batch = batch["target_batch"]
    dec_batch = batch[3]
    target_batch = dec_batch       
    #dec_lens_var = batch["target_lengths"]
    dec_lens_var = batch[4]
    #print('*******DEC BATCH*******', dec_lens_var) ----- Tensor("input_x_4:0", shape=(32, 1), dtype=int64)
    #max_dec_len = max(dec_lens_var)
    #max_dec_len = tf.math.reduce_max(dec_lens_var)
    max_dec_len = target_batch.shape[1]
    dec_padding_mask = tf.cast(sequence_mask(dec_lens_var, max_len=max_dec_len), dtype=tf.float32)
    
    return dec_batch, dec_padding_mask, max_dec_len, dec_lens_var, target_batch

def sequence_mask(sequence_length, max_len=None):
    if max_len is None:
        max_len = tf.math.reduce_max(sequence_length)
    batch_size = sequence_length.shape[0]
    seq_range = tf.range(0, max_len, dtype=tf.int32)
    seq_range_expand = tf.broadcast_to(tf.expand_dims(seq_range, 0), [batch_size, max_len])
    seq_range_expand = seq_range_expand
    #seq_length_expand = tf.broadcast_to(tf.expand_dims(sequence_length, 1), seq_range_expand.shape)
    seq_length_expand = tf.broadcast_to(sequence_length, seq_range_expand.shape)
    return seq_range_expand < seq_length_expand

def print_custum(emotion,dial,ref,hyp_g,hyp_b):
    print("emotion:{}".format(emotion))
    print("Context:{}".format(dial))
    #print("Topk:{}".format(hyp_t))
    print("Beam: {}".format(hyp_b))
    print("Greedy:{}".format(hyp_g))
    print("Ref:{}".format(ref))
    print("----------------------------------------------------------------------")
    print("----------------------------------------------------------------------")

def count_parameters(model):
    return sum(np.prod(p.get_shape()) for p in model.trainable_weights)

def make_infinite(dataloader):
    while True:
        for x in dataloader:
            yield x

def top_k_top_p_filtering(logits, top_k=0, top_p=0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (..., vocabulary size)
            top_k >0: keep only top k tokens with highest probability (top-k filtering).
            top_p >0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
    """
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][:, -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[:, 1:] = tf.identity(sorted_indices_to_remove[:, :-1])
        sorted_indices_to_remove[:, 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

In [205]:
import os
import math
import time
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from copy import deepcopy
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers

class Encoder(layers.Layer):
    def __init__(self, embedding_size, hidden_size, num_layers, num_heads, total_key_depth, total_value_depth,
    filter_size, max_length=1000, input_dropout=0.0, layer_dropout=0.0, attention_dropout=0.0, 
    relu_dropout=0.0, use_mask=False, universal=False):

        super(Encoder, self).__init__()
        self.universal = universal
        self.num_layers = num_layers
        self.timing_signal = _gen_timing_signal(max_length, hidden_size)
        
        if(self.universal):  
            ## for t
            self.position_signal = _gen_timing_signal(num_layers, hidden_size)

        params =(hidden_size, 
                 total_key_depth or hidden_size,
                 total_value_depth or hidden_size,
                 filter_size, 
                 num_heads, 
                 _gen_bias_mask(max_length) if use_mask else None,
                 layer_dropout, 
                 attention_dropout, 
                 relu_dropout)
        
        self.embedding_proj = layers.Dense(hidden_size, use_bias = False)
        if(self.universal):
            self.enc = EncoderLayer(*params)
        else:
            self.enc = [EncoderLayer(*params) for _ in range(num_layers)]
        
        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)
        self.input_dropout = layers.Dropout(input_dropout)

    def __call__(self, inputs, mask, training=True): #ADHOC = TRUE
        #Add input dropout
        
        inputs = tf.cast(inputs, dtype = tf.float32) #inputs dim (32, 38)
        x = self.input_dropout(inputs)
        
        # Project to hidden size
        x = self.embedding_proj(x) #x dim [32, 100]
        
        if(self.universal):
            for l in range(self.num_layers):
                x += tf.cast(self.timing_signal[:, :inputs.shape[1], :], inputs.dtype)
                x += tf.cast(tf.tile(tf.expand_dims(self.position_signal[:, l, :], 1), [1,inputs.shape[1],1]), inputs.dtype)
                x = self.enc(x, mask=mask, training=training)
            y = self.layer_norm(x)
        else:
            x += tf.cast(self.timing_signal[:, :inputs.shape[1], :], inputs.dtype)   
            for i in range(self.num_layers):
                x = self.enc[i](x, mask, training=training)
            y = self.layer_norm(x)
        return y

class Decoder(layers.Layer):
    def __init__(self, embedding_size, hidden_size, num_layers, num_heads, total_key_depth, total_value_depth,
                 filter_size, max_length=1000, input_dropout=0.0, layer_dropout=0.0, 
                 attention_dropout=0.0, relu_dropout=0.0, universal=False):
        super(Decoder, self).__init__()
        self.universal = universal
        self.num_layers = num_layers
        self.timing_signal = _gen_timing_signal(max_length, hidden_size)
        
        if(self.universal):  
            self.position_signal = _gen_timing_signal(num_layers, hidden_size)

        self.mask = _get_attn_subsequent_mask(max_length)

        params =(hidden_size, 
                 total_key_depth or hidden_size,
                 total_value_depth or hidden_size,
                 filter_size, 
                 num_heads, 
                 _gen_bias_mask(max_length), # mandatory
                 layer_dropout, 
                 attention_dropout, 
                 relu_dropout)
        
        if(self.universal):
            self.dec = DecoderLayer(*params)
        else:
            self.dec = [DecoderLayer(*params) for l in range(num_layers)]
        
        self.embedding_proj = layers.Dense(hidden_size, use_bias = False)
        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)
        self.input_dropout = layers.Dropout(input_dropout)


    def __call__(self, inputs, encoder_output, mask, training=True):#ADHOC = TRUE
        mask_src, mask_trg = mask
        dec_mask = tf.math.greater(mask_trg + self.mask[:, :mask_trg.shape[-1], :mask_trg.shape[-1]], 0)
        #Add input dropout
        x = self.input_dropout(inputs)
        x = self.embedding_proj(x)
            
        if(self.universal):
            x += tf.cast(self.timing_signal[:, :inputs.shape[1], :], inputs.dtype)
            for l in range(self.num_layers):
                x += tf.cast(tf.tile(tf.expand_dims(self.position_signal[:, l, :], 1), [1,inputs.shape[1],1]), inputs.dtype)
                x, _, attn_dist, _ = self.dec((x, encoder_output, [], (mask_src,dec_mask)),training=training)
            y = self.layer_norm(x)
        else:
            # Add timing signal
            x += tf.cast(self.timing_signal[:, :inputs.shape[1], :], inputs.dtype) 
            # Run decoder
            for i in range(self.num_layers):
                x, _, attn_dist, _ = self.dec[i]((x, encoder_output, [], (mask_src,dec_mask)),training=training)
            y = self.layer_norm(x)
        return y, attn_dist

class MulDecoder(layers.Layer):
    def __init__(self, expert_num,  embedding_size, hidden_size, num_layers, num_heads, total_key_depth, total_value_depth,
                 filter_size, max_length=1000, input_dropout=0.0, layer_dropout=0.0, 
                 attention_dropout=0.0, relu_dropout=0.0):
        super(MulDecoder, self).__init__()
        self.num_layers = num_layers
        self.timing_signal = _gen_timing_signal(max_length, hidden_size)
        self.mask = _get_attn_subsequent_mask(max_length)

        params =(hidden_size, 
                 total_key_depth or hidden_size,
                 total_value_depth or hidden_size,
                 filter_size, 
                 num_heads, 
                 _gen_bias_mask(max_length), # mandatory
                 layer_dropout, 
                 attention_dropout, 
                 relu_dropout)
        self.experts = [DecoderLayer(*params) for e in range(expert_num)]
        self.dec = [DecoderLayer(*params) for l in range(num_layers)]
        
        self.embedding_proj = layers.Dense(hidden_size, use_bias = False)
        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)
        self.input_dropout = layers.Dropout(input_dropout)
    
    def __call__(self, inputs, encoder_output, mask, attention_epxert, training=True):#ADHOC = TRUE
        mask_src, mask_trg = mask
        dec_mask = tf.math.greater(tf.cast(mask_trg, tf.uint8) + self.mask[:, :mask_trg.shape[-1], :mask_trg.shape[-1]], 0)
        #Add input dropout
        x = self.input_dropout(inputs)
        x = self.embedding_proj(x)
        # Add timing signal
        x += tf.cast(self.timing_signal[:, :inputs.shape[1], :], inputs.dtype)
        expert_outputs = []

        #compute experts
        #TODO forward all experts in parrallel
        for i, expert in enumerate(self.experts):
            expert_out , _, attn_dist, _ = expert((x, encoder_output, [], (mask_src,dec_mask)), training=training)
            expert_outputs.append(expert_out)
        x = tf.stack(expert_outputs, axis=1) #(batch_size, expert_number, len, hidden_size)
        x = attention_epxert * x
        x = tf.reduce_sum(x, axis=1)#(batch_size, len, hidden_size)
        # Run decoder
        for i in range(self.num_layers):
            x, _, attn_dist, _ = self.dec[i]((x, encoder_output, [], (mask_src,dec_mask)), training=training)
        # Final layer normalization
        y = self.layer_norm(x)
        return y, attn_dist

class Generator(layers.Layer):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = layers.Dense(vocab)
        self.p_gen_linear = layers.Dense(1)

    def __call__(self, x, attn_dist=None, enc_batch_extend_vocab=None, extra_zeros=None, temp=1, beam_search=False, attn_dist_db=None):
        logit = self.proj(x)
        return tf.nn.log_softmax(logit, axis = -1)

class Transformer_experts(layers.Layer):
    def __init__(self, vocab, decoder_number, model_file_path=None, is_eval=False, load_optim=False):
        super(Transformer_experts, self).__init__()
        self.vocab = vocab
        self.vocab_size = vocab.n_words

        self.embedding = Embeddinglayer(self.vocab.n_words, 100)#修改
        self.encoder = Encoder(100, 100, num_layers=2, num_heads=2, total_key_depth=40, total_value_depth=40,
                                filter_size=50,universal=False)
        self.decoder_number = decoder_number
        ## multiple decoders
        self.decoder = MulDecoder(decoder_number, 100, 100,  num_layers=2, num_heads=2, 
                                    total_key_depth=40,total_value_depth=40,filter_size=50)
 
        self.decoder_key = layers.Dense(decoder_number, use_bias=False)

        self.generator = Generator(100, self.vocab_size)
        self.emoji_embedding = layers.Dense(100, use_bias=False)
        self.criterion = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction='none', label_smoothing=0.1)
        
        self.attention_activation =  layers.Activation('sigmoid') #nn.Softmax()
        
        self.optimizer = NoamOpt(100, 1, 8000, tf.keras.optimizers.Adam(lr=0, beta_1=0.9, beta_2=0.98, epsilon=1e-9))
    
    def __call__(self, batch, training=True): #ADHC training=True
        enc_batch, _, _, enc_batch_extend_vocab, extra_zeros, _, _ = get_input_from_batch(batch)
        dec_batch, _, _, _, _ = get_output_from_batch(batch)
        ## Encode
        mask_src = tf.expand_dims(tf.math.equal(enc_batch, 1), axis = 1)
        emb_mask = self.embedding(batch[2]) #"mask_input"
        #print('Encoder Input: ', (self.embedding(enc_batch)+emb_mask).shape)
        encoder_outputs = self.encoder(self.embedding(enc_batch)+emb_mask, mask_src, training=training)
        
        ## Attention over decoder
        q_h = encoder_outputs[:,0]
        #q_h = encoder_outputs[:,0]
        logit_prob = self.decoder_key(q_h) #(bsz, num_experts)
        attention_parameters = self.attention_activation(logit_prob)
        attention_parameters = tf.expand_dims(tf.expand_dims(attention_parameters, axis=-1), axis=-1) # (batch_size, expert_num, 1, 1)
        # Decode 
        sos_token = tf.expand_dims(tf.cast([3] * enc_batch.shape[0], dtype=tf.int32), axis=1)
        dec_batch_shift = tf.concat((sos_token, dec_batch[:, :-1]), axis=1)
        mask_trg = tf.expand_dims(tf.math.equal(dec_batch_shift, 1), axis=1)
       
        pre_logit, attn_dist = self.decoder(self.embedding(dec_batch_shift), encoder_outputs, (mask_src,mask_trg), attention_parameters, training=training)
        ## compute output dist
        logit = self.generator(pre_logit, attn_dist, None, extra_zeros, attn_dist_db=None)
        return logit, logit_prob
    
    def decoder_greedy(self, batch, max_dec_step=30, training=False):
        enc_batch, _, _, enc_batch_extend_vocab, extra_zeros, _, _ = get_input_from_batch(batch)
        mask_src = tf.expand_dims(tf.math.equal(enc_batch, 1), axis = 1)
        emb_mask = self.embedding(batch[2]) #"mask_input"
        encoder_outputs = self.encoder(self.embedding(enc_batch)+emb_mask, mask_src, training=training)
        ## Attention over decoder
        q_h = encoder_outputs[:,0]
        #q_h = encoder_outputs[:,0]
        logit_prob = self.decoder_key(q_h)    
        attention_parameters = self.attention_activation(logit_prob)
        attention_parameters = tf.expand_dims(tf.expand_dims(attention_parameters, axis=-1), axis=-1) # (batch_size, expert_num, 1, 1)

        ys = tf.cast(tf.fill([1, 1], 3), dtype=tf.int32)#修改int64
        mask_trg = tf.expand_dims(tf.math.equal(ys, 1), axis=1)
        decoded_words = []
        for i in range(max_dec_step+1):
            out, attn_dist = self.decoder(self.embedding(ys),encoder_outputs, (mask_src,mask_trg), attention_parameters, training=training)
            logit = self.generator(out,attn_dist,enc_batch_extend_vocab, extra_zeros, attn_dist_db=None)
            next_word = tf.math.argmax(logit[:, -1], axis = 1)
            decoded_words.append(['<EOS>' if ni.item() == 2 else self.vocab.index2word[ni] for ni in tf.reshape(next_word, -1).numpy()])
            next_word = next_word[0]
            ys = tf.concat([ys, tf.cast(tf.fill([1, 1], next_word), dtype=tf.int32)], axis=1)#修改int64
            mask_trg = tf.expand_dims(tf.math.equal(ys, 1), axis=1)

        sent = []
        for _, row in enumerate(np.transpose(decoded_words)):
            st = ''
            for e in row:
                if e == '<EOS>': 
                    break
                else: 
                    st+= e + ' '
            sent.append(st)
        return sent
    
    def decoder_topk(self, batch, max_dec_step=30, training=False):
        enc_batch, _, _, enc_batch_extend_vocab, extra_zeros, _, _ = get_input_from_batch(batch)
        mask_src = tf.expand_dims(tf.math.equal(enc_batch, 1), axis = 1)
        emb_mask = self.embedding(batch[2]) #"mask_input"
        encoder_outputs = self.encoder(self.embedding(enc_batch)+emb_mask, mask_src, training=training)
        ## Attention over decoder
        q_h = encoder_outputs[:,0]
        #q_h = encoder_outputs[:,0]
        logit_prob = self.decoder_key(q_h)

        attention_parameters = self.attention_activation(logit_prob)
        attention_parameters = tf.expand_dims(tf.expand_dims(attention_parameters, axis=-1), axis=-1) # (batch_size, expert_num, 1, 1)

        ys = tf.cast(tf.fill([1, 1], 3), dtype=tf.int32)#修改int64
        mask_trg = tf.expand_dims(tf.math.equal(ys, 1), axis=1)
        decoded_words = []
        for i in range(max_dec_step+1):
            out, attn_dist = self.decoder(self.embedding(ys),encoder_outputs, (mask_src,mask_trg), attention_parameters, training=training)
            logit = self.generator(out,attn_dist,enc_batch_extend_vocab, extra_zeros, attn_dist_db=None)
            filtered_logit = top_k_top_p_filtering(logit[:, -1], top_k=3, top_p=0, filter_value=-float('Inf'))
            # Sample from the filtered distribution
            next_word = tf.squeeze(tf.random.categorical(tf.nn.softmax(filtered_logit, axis=-1), 1))
            decoded_words.append(['<EOS>' if ni.item() == 2 else self.vocab.index2word[ni] for ni in tf.reshape(next_word, -1).numpy()])
            next_word = next_word[0]

            ys = tf.concat([ys, tf.cast(tf.fill([1, 1], next_word), dtype=tf.int32)], axis=1)#修改int64
            mask_trg = tf.expand_dims(tf.math.equal(ys, 1), axis=1)

        sent = []
        for _, row in enumerate(np.transpose(decoded_words)):
            st = ''
            for e in row:
                if e == '<EOS>':
                    break
                else:
                    st+= e + ' '
            sent.append(st)
        return sent

class ACT_basic(layers.Layer):
    def __init__(self,hidden_size):
        super(ACT_basic, self).__init__()
        self.sigma = layers.Activation('sigmoid')
        self.p = layers.Dense(1, bias_initializer='ones')
        self.threshold = 1 - 0.1
    
    def __call__(self, state, inputs, fn, time_enc, pos_enc, max_hop, encoder_output=None, decoding=False):
        # init_hdd
        ## [B, S]
        halting_probability = tf.zeros(inputs.shape[0],inputs.shape[1])
        ## [B, S
        remainders = tf.zeros(inputs.shape[0],inputs.shape[1])
        ## [B, S]
        n_updates = tf.zeros(inputs.shape[0],inputs.shape[1])
        ## [B, S, HDD]
        previous_state = tf.zeros(inputs.shape)
        step = 0
        # for l in range(self.num_layers):
        while( ((halting_probability<self.threshold) & (n_updates < max_hop)).byte().any()):
            # Add Timing Signal
            state = state + tf.cast(time_enc[:, :inputs.shape[1], :], inputs.dtype)
            state = state + tf.cast(tf.tile(tf.expand_dims(pos_enc[:, step, :], 1), [1,inputs.shape[1],1]), inputs.dtype)

            p = tf.squeeze(self.sigma(self.p(state)), axis=-1)
            # Mask for inputs which have not halted yet
            still_running = tf.cast((halting_probability < 1.0), dtype=tf.float32)
            # Mask of inputs which halted at this step
            new_halted = tf.cast((halting_probability + p * still_running > self.threshold), dtype=tf.float32) * still_running
            # Mask of inputs which haven't halted, and didn't halt this step
            still_running = tf.cast((halting_probability + p * still_running <= self.threshold), dtype=tf.float32) * still_running
            # Add the halting probability for this step to the halting
            # probabilities for those input which haven't halted yet
            halting_probability = halting_probability + p * still_running
            # Compute remainders for the inputs which halted at this step
            remainders = remainders + new_halted * (1 - halting_probability)
            # Add the remainders to those inputs which halted at this step
            halting_probability = halting_probability + new_halted * remainders
            # Increment n_updates for all inputs which are still running
            n_updates = n_updates + still_running + new_halted
            # Compute the weight to be applied to the new state and output
            # 0 when the input has already halted
            # p when the input hasn't halted yet
            # the remainders when it halted this step
            update_weights = p * still_running + new_halted * remainders
            if(decoding):
                state, _, attention_weight = fn((state,encoder_output,[]))
            else:
                # apply transformation on the state
                state = fn(state)
            # update running part in the weighted state and keep the rest
            previous_state = ((state * tf.expand_dims(update_weights, axis=-1)) + (previous_state * (1 - tf.expand_dims(update_weights, axis=-1))))
            if(decoding):
                if(step==0): 
                    previous_att_weight = tf.zeros(attention_weight.shape)## [B, S, src_size]
            previous_att_weight = ((attention_weight * tf.expand_dims(update_weights, axis=-1)) + (previous_att_weight * (1 - tf.expand_dims(update_weights, axis=-1))))
            ## previous_state is actually the new_state at end of hte loop 
            ## to save a line I assigned to previous_state so in the next 
            ## iteration is correct. Notice that indeed we return previous_state
            step+=1
        if(decoding):
            return previous_state, previous_att_weight, (remainders,n_updates)
        else:
            return previous_state, (remainders,n_updates)

In [218]:
model = Transformer_experts(vocab,decoder_number=program_number)
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)#修改
#optimizer = NoamOpt(100, 1, 8000, tf.keras.optimizers.Adam(lr=0, beta_1=0.9, beta_2=0.98, epsilon=1e-9))
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

training_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')#修改
testing_loss = tf.keras.metrics.Mean('validation_loss', dtype=tf.float32)
testing_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('validation_accuracy')#修改

#@tf.function 
def train_step(input_x, training = True):
    dec_batch, _, _, _, _ = get_output_from_batch(input_x)
    if training:
        with tf.GradientTape() as tape:
            logit, logit_prob = model(input_x, training)
            train_loss = criterion(tf.reshape(dec_batch, -1), tf.reshape(logit, [-1, logit.shape[-1]])) + criterion(tf.cast(input_x[6], dtype=tf.int32), logit_prob)
        gradients = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        #train_pred_program = np.argmax(logit_prob.numpy(), axis=1)
        training_loss.update_state(train_loss)
        training_accuracy.update_state(input_x[6], logit_prob)
    else:
        logit, logit_prob = model(input_x, training)
        test_loss = criterion(tf.reshape(dec_batch, -1), tf.reshape(logit, [-1, logit.shape[-1]])) + criterion(tf.cast(input_x[6], dtype=tf.int32), logit_prob)
        #test_pred_program = np.argmax(logit_prob.numpy(), axis=1)
        testing_loss.update_state(test_loss)
        testing_accuracy.update_state(input_x[6], logit_prob)

In [220]:
for i in range(10):
    train_step(next(data_iter), training = True)
    print('step {}'.format(str(i+1)))

step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10


In [215]:
training=True
model = Transformer_experts(vocab,decoder_number=program_number)
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)#修改
#optimizer = NoamOpt(100, 1, 8000, tf.keras.optimizers.Adam(lr=0, beta_1=0.9, beta_2=0.98, epsilon=1e-9))
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

training_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')#修改
testing_loss = tf.keras.metrics.Mean('validation_loss', dtype=tf.float32)
testing_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('validation_accuracy')#修改

for i in range(20):
    cur_input=next(data_iter)
    dec_batch, _, _, _, _ = get_output_from_batch(cur_input)
    if training:
        with tf.GradientTape() as tape:
            logit, logit_prob = model(cur_input, training)
            train_loss = criterion(tf.reshape(dec_batch, -1), tf.reshape(logit, [-1, logit.shape[-1]])) + criterion(tf.cast(input_x[6], dtype=tf.int32), logit_prob)
        gradients = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        #train_pred_program = np.argmax(logit_prob.numpy(), axis=1)
        training_loss.update_state(train_loss)
        training_accuracy.update_state(cur_input[6], logit_prob)
        print('step {}'.format(str(i+1)), 'curr loss: ', train_loss.numpy())
    else:
        logit, logit_prob = model(cur_input, training)
        test_loss = criterion(tf.reshape(dec_batch, -1), tf.reshape(logit, [-1, logit.shape[-1]])) + criterion(tf.cast(input_x[6], dtype=tf.int32), logit_prob)
        #test_pred_program = np.argmax(logit_prob.numpy(), axis=1)       
        testing_loss.update_state(test_loss)
        testing_accuracy.update_state(cur_input[6], logit_prob)

step 1 curr loss:  14.215617
step 2 curr loss:  14.658287
step 3 curr loss:  14.073604
step 4 curr loss:  14.045111
step 5 curr loss:  14.289752
step 6 curr loss:  13.947664
step 7 curr loss:  13.951778
step 8 curr loss:  13.64507
step 9 curr loss:  13.47399
step 10 curr loss:  13.636381
step 11 curr loss:  13.534122
step 12 curr loss:  13.414142
step 13 curr loss:  13.393187
step 14 curr loss:  13.125283
step 15 curr loss:  13.144095
step 16 curr loss:  13.342746
step 17 curr loss:  13.382704
step 18 curr loss:  13.081802
step 19 curr loss:  12.83449
step 20 curr loss:  13.311848
